# Testing convergence of MCMC method for evidence value calculation

In [1]:
import numpy as np


In [2]:

def cposterior_full(d, s, Nw, N2, beta, sum1, sum2):
    ''' Full log-posterior kernel for MCMC sampling

        Arguments:

        d - current value for delta
        s - current value for sigma
        Nw - total signal size
        N2 - size of second segment
        beta - parameter for laplace prior
        sum1 - sum of amplitudes squared for first segment
        sum2 - sum of amplitudes squared for second segment
    '''
    
    if d <= 0 or s <= 0:
        return -1e+308
    
    # Jeffreys' prior for sigma
    dpriors = -np.log(s)

    # Laplace prior for delta
    dpriord = -np.log(beta) - abs(d-1)/beta

    post = dpriord +  dpriors - Nw*np.log(s)-0.5*N2*np.log(d)
    post = post - sum1/(2*(s**2)) - sum2/(2*d*(s**2))

    return post


In [3]:
def mcmc(p0, mcburn, mciter, beta, N, N2, sum1, sum2):
    ''' Run MCMC

        Arguments:

        mcburn - burn-in period for chain
        mciter - number of points to sample
        p0 - posterior under H0
        beta - parameter for Laplace prior
        N - total signal size
        N2 - size of second segment
        sum1 - sum of amplitude squared for the first segment
        sum2 - sum of amplitude squared for the second segment

    '''
    
    dcur = (sum2 / (N2-1))/(sum1 / (N-N2-1))
    scur = np.sqrt(sum1 / (N-N2-1))

    # Standard deviations and covariance for random-walk candidates distributions
    dvar = (dcur / 3) ** 2
    svar = (scur / 3) ** 2
    cov = 0.0
    
    dcur = abs(dcur + np.random.normal(0, dvar))
    scur = abs(scur + np.random.normal(0, svar))

    pcur = cposterior_full(dcur, scur, N, N2, beta, sum1, sum2)
    
    
    # Parameters for adaptive MH
    sd = (2.4*2.4)/2.0
    eps = 1e-30

    # Starting point for adaptive MH
    t0 = 500
    
    dmean = 0.0
    smean = 0.0
    sumdsq = 0.0
    sumssq = 0.0
    cov0 = 0.0
    accept = 0
    for i in range(t0):
        
        # Generate candidates
        u1 = np.random.normal(0, 1)
        dcand = dcur + u1*np.sqrt(dvar)
        
        if dcand > 0:
            # Calculates full posterior
            pcand = cposterior_full(dcand, scur, N, N2, beta, sum1, sum2)

            # Acceptance ratio
            #a = Exp(pcand - pcur) * Exp(scand - scur)
            a = (pcand - pcur)

            if np.log(np.random.uniform()) < a:
                dcur = dcand
                pcur = pcand
                accept = accept + 1
            #endif        
        
        u2 = np.random.normal(0, 1)
        scand = abs(scur + np.sqrt(svar)*u2)
        
        if scand > 0:
            # Calculates full posterior
            pcand = cposterior_full(dcur, scand, N, N2, beta, sum1, sum2)

            # Acceptance ratio
            #a = Exp(pcand - pcur) * Exp(scand - scur)
            a = (pcand - pcur)

            if np.log(np.random.uniform()) < a:
                scur = scand
                pcur = pcand
                accept = accept + 1
            #endif
        
        dmean = dmean + dcur 
        smean = smean + scur
        cov0 = cov0 + dcur*scur 
        sumdsq = sumdsq + dcur*dcur 
        sumssq = sumssq + scur*scur 
        
    #endfor
    
    assert accept > 0
    tmp = dmean*dmean/t0
    tmp = sumdsq - tmp
    dvar = tmp/(t0-1.0)
    tmp = smean*smean/t0
    tmp = sumssq - tmp
    svar = tmp/(t0-1.0)
    
    if svar < 0:
        print(accept)
        print((smean*smean)/t0)
        print([sumssq, smean, t0])
        print(svar)
    
    cov = (1/(t0-1))*(cov0 - dmean*smean/t0)
    rho = cov/(np.sqrt(dvar*svar))
    dmean = dmean / t0
    smean = smean / t0    
    t = t0

    accept = 0
    for i in range(mcburn):
        
        # Generate candidates
        u1 = np.random.normal(0, 1)
        u2 = np.random.normal(0, 1)
        if abs(rho) > 1:
            print([dvar, svar, cov])
            assert abs(rho) <= 1
        u2 = rho * u1 + (1 - rho) * u2
        
        dcand = dcur + u1*np.sqrt(dvar)
        #scand = scur + (cov/np.sqrt(dvar))*u1 + np.sqrt(svar - (cov*cov)/(dvar))*u2
        scand = scur + u2*np.sqrt(svar)
        
        if dcand > 0 and scand > 0:
            # Calculates full posterior
            pcand = cposterior_full(dcand, scand, N, N2, beta, sum1, sum2)

            # Acceptance ratio
            #a = Exp(pcand - pcur) * Exp(scand - scur)
            a = (pcand - pcur)

            if np.log(np.random.uniform()) < a:
                scur = scand
                dcur = dcand
                pcur = pcand
                accept = accept + 1
            #endif
            
        # Updating covariance matrix
        dmeanant = dmean
        smeanant = smean
        dmean = (t*dmeanant + dcur) / (t + 1)
        smean = (t*smeanant + scur) / (t + 1)            
        dvar =  ((t-1.0)*dvar)/t + (sd/t)*(t*dmeanant*dmeanant - (t+1)*dmean*dmean + dcur*dcur + eps)
        
        if ((t-1.0)*svar)/t + (sd/t)*(t*smeanant*smeanant - (t+1)*smean*smean + scur*scur + eps) < 0:
            print([t, svar, sd, smeanant, smean, scur, eps])        
        
        svar =  ((t-1.0)*svar)/t + (sd/t)*(t*smeanant*smeanant - (t+1)*smean*smean + scur*scur + eps)
        cov = ((t-1.0)*cov)/t + (sd/t)*(t*dmeanant*smeanant - (t+1)*dmean*smean + dcur*scur)

        rho = cov/(np.sqrt(dvar*svar))
        t = t + 1
        assert dvar > 0
        assert svar > 0
            
    #endfor

    accept = 0
    ev = 0
    sample = []
    for i in range(mciter):
        # Generate candidates
        u1 = np.random.normal(0, 1)
        u2 = np.random.normal(0, 1)
        u2 = rho*u1 + (1-rho)*u2
        
        dcand = dcur + u1*np.sqrt(dvar)
        #scand = scur + (cov/np.sqrt(dvar))*u1 + np.sqrt(svar - (cov*cov)/(dvar))*u2
        scand = scur + np.sqrt(svar)*u2

        if dcand > 0 and scand > 0:
            # Calculates full posterior
            pcand = cposterior_full(dcand, scand, N, N2, beta, sum1, sum2)

            # Acceptance ratio
            a = pcand - pcur

            if np.log(np.random.uniform()) < a:
                dcur = dcand
                scur = scand
                pcur = pcand
                accept = accept + 1
            #endif
        sample.append([dcur, scur])
        if pcur > p0:
            ev = ev + 1
    
    ev = 1 - ev / mciter

    return sample, ev, accept / mciter


In [4]:
# Simulates signal: one change point
n1 = 500000
n2 = 500000
N = n1 + n2
stdnoise = [1]
delta = [1, 1.1, 1.5]

mciter = [1000, 10000, 100000]
beta = 1

M = 4
final = []
cont = 0
total = len(stdnoise) * len(delta) * len(mciter) 

# Set random seed for replication purposes
np.random.seed(123456)
for sn in stdnoise:
    for d in delta:
        signal = np.concatenate([np.random.normal(0, sn, [n1, 1]), np.random.normal(0, np.sqrt(d)*sn, [n2, 1])])
        sum1 = sum(signal[:n1]**2)
        sum2 = sum(signal[n1:]**2)
        s0 = np.sqrt((sum1 + sum2)/(N + 1.))
        p0 = cposterior_full(1.0, s0, N, n2, beta, sum1, sum2)
        for mn in mciter:
            measures = []
            for i in range(M):
                sample, ev, acc = mcmc(p0, mn, mn, beta, n1+n2, n2, sum1, sum2)
                dsample = np.asarray([s[0] for s in sample])
                ssample = np.asarray([s[1] for s in sample])
                davg = dsample.mean()
                savg = ssample.mean()
                dvar = dsample.var()*(mn/(mn-1))
                svar = ssample.var()*(mn/(mn-1))
                measures.append([davg, savg, dvar, svar, ev, acc])
                
            minev = min([m[4] for m in measures])
            maxev = max([m[4] for m in measures])
            dhat = (1/M)*sum([m[0] for m in measures])
            shat = (1/M)*sum([m[1] for m in measures])
            dB = (1 / (M-1))*sum([(m[0] - dhat)**2 for m in measures])
            sB = (1 / (M-1))*sum([(m[1] - shat)**2 for m in measures])
            dW = (1/M)*sum([m[2] for m in measures])
            sW = (1/M)*sum([m[3] for m in measures])
            accavg = (1/M)*sum([m[5] for m in measures])
            dVhat = (mn-1)/mn * dW + dB + dB / M
            sVhat = (mn-1)/mn * sW + sB + sB / M
            dRhat = dVhat / dW
            sRhat = sVhat / sW
            final.append([n1, sn, d, mn, minev, maxev, accavg, dhat, shat, dRhat, sRhat])
            print("Run " + str(cont+1) + " of " + str(total))
            cont = cont + 1


Run 1 of 9
Run 2 of 9
Run 3 of 9
Run 4 of 9
Run 5 of 9
Run 6 of 9
Run 7 of 9
Run 8 of 9
Run 9 of 9


In [5]:
import pandas as pd
df = pd.DataFrame(final, columns = ['nsignal', 'stdnoise', 'delta', 'mciter', 'minev', 'maxev', 'accavg', 'dhat', 'shat', 'dRhat', 'sRhat'])

# Generate latex code
print(df.to_latex())

\begin{tabular}{lrrrrrrrrrrr}
\toprule
{} &  nsignal &  stdnoise &  delta &  mciter &    minev &    maxev &    accavg &      dhat &      shat &     dRhat &     sRhat \\
\midrule
0 &   500000 &         1 &    1.0 &    1000 &  0.96100 &  1.00000 &  0.023250 &  1.000344 &  1.000005 &  2.236312 &  2.130287 \\
1 &   500000 &         1 &    1.0 &   10000 &  0.97260 &  0.98510 &  0.100125 &  1.000527 &  1.000023 &  1.009749 &  1.004708 \\
2 &   500000 &         1 &    1.0 &  100000 &  0.97617 &  0.98112 &  0.136043 &  1.000595 &  0.999983 &  1.000193 &  1.000106 \\
3 &   500000 &         1 &    1.1 &    1000 &  0.00000 &  0.00000 &  0.018500 &  1.098619 &  0.999929 &  1.328783 &  1.111929 \\
4 &   500000 &         1 &    1.1 &   10000 &  0.00000 &  0.00000 &  0.052250 &  1.097646 &  1.000127 &  1.010574 &  1.011942 \\
5 &   500000 &         1 &    1.1 &  100000 &  0.00000 &  0.00000 &  0.107863 &  1.097803 &  1.000070 &  1.000113 &  1.000146 \\
6 &   500000 &         1 &    1.5 &    1000 &  0